In [ ]:
!pip install transformers
!pip install torch
!pip install fuzzywuzzy



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile app.py

import pandas as pd
import streamlit as st
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from fuzzywuzzy import fuzz

# Read the dataset
df = pd.read_csv('question_answer.csv')

# Set up the model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Define the get_answer function
def get_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer

# Streamlit app
st.title("Call Agent Chatbot")
st.write("Hello! I'm here to answer your questions about Gardening.")

conversation_history = []

user_input = st.text_input("Type your question here:")

if st.button("Ask"):
    if user_input:
        conversation_history.append(("User", user_input))

        # Check if the question is in the dataset
        match = None
        max_score = -1
        for i, question in enumerate(df['Question']):
            score = fuzz.token_sort_ratio(user_input, question)
            if score > max_score:
                max_score = score
                match = df.iloc[i]

        # If the score is high enough, display the answer
        if max_score > 70:
            context = match['Answer']
            answer = get_answer(user_input, context)
        # If the score is not high enough, call the agent
        else:
            answer = "I'm sorry ,my data are not trained for that question.I will Call the Agent"

        conversation_history.append(("Chatbot", answer))

        for speaker, message in conversation_history:
            if speaker == "User":
                st.markdown(f"> {speaker}: *{message}*")
            else:
                st.markdown(f"{speaker}: {message}")

    else:
        st.write("Please type a question.")

Overwriting app.py


In [ ]:

!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

!streamlit run app.py & npx localtunnel --port 8501

[#######...........] - extract:localtunnel: verb lock using /root/.npm/_locks/s


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.172.129.100:8501

npx: installed 22 in 5.594s
your url is: https://fuzzy-emus-do.loca.lt
/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
